Este codigo extrae los lfp canal por canal, opcionalmente los filtra, 
hace dowsampling y los pone en una variable y guarda la variable disco
### Pseudo código
1. saber los nombres de todos los archivos de lfp de esa sesion 
2. hacer un loop for con el numero
3. dentro del loop> cargar un canal-archivo
4. No vamos a filtrar aqui (opcional), comentar o descomentar la linea de filtrado
5. Hacer dowsampling del lfp
6. El resultado se pone en una dataframe panda
7. Guardar el archivo

notas, se debe conservar el numero de canal , y otras informaciones 

In [ ]:
import sys
sys.path.append('/home/sergio/Documentos/MemoriaPractII/Open Ephys/analysis-tools-master')

In [ ]:
%pylab
import pickle
from glob import glob
import OpenEphys as OE
import os
import scipy
import pandas as pd

#import statistics as stats

In [ ]:
# definir el root path del directorio donde estan los archivos
#rootpath = '/home/romulo/DATA/FLORENCIA/test/'
#rootpath = '/home/romulo/DATA/PATO/SERT_ALCE01_2016-09-13_17-07-18/'
# rootpath = '/home/romulo/DATA/FLORENCIA/daniel_2016_12_28/'
# rootpath = '/home/romulo/DATA/FLORENCIA/Kobe_2016_06_01/'
#rootpath = '/home/romulo/DATA/PATO/SERT_A01_2016_07_21/'
# rootpath = '/home/romulo/DATA/FLORENCIA/daniel_2016_12_28/'
#rootpath = '/home/romulo/DATA/Carolina/R02/RAT02_SP_7_2017_01_11_18_46_49/'
#rootpath = '/home/romulo/DATA/Carolina/RR01/EC01_RRR01_SP_2650uA_2017-12-01_17-01-13/'
#rootpath = '/home/romulo/DATA/Carolina/C05/EC01_RC05_SP_10100uA_20171214_170009/'
#rootpath = '/media/romulo/USB DISK/CARO/RAT02_SP_7__2017-01-11_18-46-49/'

#rootpath='/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_3500uA_20171215/'
#rootpath='/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_3500uA_L-Dopa_20171215/'
#rootpath='/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_4800uA_20171221/'
#rootpath='/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_1240uA_20171212/'
#rootpath='/home/romulo/DATA/Carolina/N03/E01_RNA03_SP02_500mV_20170923/'
# rootpath = '/home/romulo/DATA/Carolina/RR01/EC01_RR01_SP_6000uA_20171229/' not good
#rootpath = '/home/romulo/DATA/Carolina/RR01/EC01_RRR01_SP_12600uA__20180103/'

In [ ]:
#DONErootpath = '/home/sergio/Documentos/MemoriaPractII/DatosCaro/RR01/EC01_RRR01_SP_12600uA__2018-01-03_13-09-47/'
rootpath = '/home/sergio/Documentos/MemoriaPractII/DatosCaro/RAT02/RAT02_SP_7__2017-01-11_18-46-49/'
#DONErootpath = '/home/sergio/Documentos/MemoriaPractII/DatosCaro/RRR03/EC01_RRR03_SP_3500uA_2017-12-15_11-45-36/'
#DONErootpath = '/home/sergio/Documentos/MemoriaPractII/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/'
fullpath = rootpath +'*CH*.continuous'
lista = glob(fullpath)
lista = sorted(lista)
lista

In [ ]:
# obtain sampling rate from first channel to obtain samplin freq
i = 0
data = OE.loadContinuous(lista[i])
ad = data['data']
orSampleRate = int(data['header']['sampleRate'])
Nyquist = orSampleRate/2

In [ ]:
print(orSampleRate)
print(Nyquist)
data

In [ ]:
#Grab timestamps
ts = data['timestamps']
len(ts)
aux = data['data']
print (len(ts),len(aux)/30000,(ts[-1]-ts[0])/30000)
aux

All data in the GUI is timestamped. The timestamps are represented in int64 format, representing the sample count from the start of acquisition, which can be converted to seconds by dividing into the corresponding sample rate. In the case of continuous data, the timestamp corresponding to the first sample of each processing block can be obtained with the getTimestamp() method. For spikes and events, the timestamp is embedded into the data structure itself.

El csv tiene la información por canales, y a qué área corresponde cada canal. Solo para la rata 2 la información es distinta.

Ratas 1,3,5 = 'RAT05_SP_10100uA_2017_12_14.csv'

Rata 2 = 'channel_codes_Rata_R02.csv'

In [ ]:
### filtra el lfp canal por canal
#channelcodes = 'channel_codes_florencia_2.csv'
#channelcodes = 'channel_codes_Rata_R02.csv'
#channelcodes = 'channel_codes_RRR01_CO5_RR03.csv'

##Channel codes de ratas 1,3 y 5
#channelcodes = 'RAT05_SP_10100uA_2017_12_14.csv'
##Channel codes de rata 2
channelcodes = 'channel_codes_Rata_R02.csv'
# order of the butter filter
#N = 2
# low cutoff freq
#f1 = 0.5
# high cut off freq
#f2 = 150.0
# band pass freqs expressed as Nyquist
#Wn = array([f1/Nyquist, f2/Nyquist])
# design parameters for the butter filter
#b,a = scipy.signal.butter(N, Wn, btype='bandpass', analog=False)
desiredSampleRate = 1000.0
codes_df = pd.read_csv('/home/sergio/Documentos/MemoriaPractII/DatosCaro/'+channelcodes)
df = pd.DataFrame([], columns=['Channel', 
                               'Area',
                               'OriginalSampleRate',
                               'SampleRate',
                               'Use',
                               'Timestamps at original sample rate',
                               'LFP'])
for i in range(0,len(lista)):
    #print i, lista[i]
    # lee el archivo en el i-esimo lugar de la lista y lo coloca en el diccionario data
    data = OE.loadContinuous(lista[i])
    # coloca la key data en el array aux
    aux = data['data']
    # filter the lfp signal in place
    #aux = scipy.signal.filtfilt(b, a, aux, padlen=None)
    # hace el donwsampling desde el sampling original hasta 1000 Hz
    downsamplefactor = int(orSampleRate/desiredSampleRate)
    # multiplo de downsampling factor
    multiplo_dsf = int(len(aux)/downsamplefactor)
    
    # crear array de n x downsamplefactor para promediar 
    aux2 = aux[0:downsamplefactor*multiplo_dsf].reshape(-1, downsamplefactor)
    
    # promedio y con esto el downsampling esta finalizado
    aux2 = aux2.mean(axis=1)
    
    # crear eje de tiempo para la señal down-sampleada
    t2 = np.arange(1/desiredSampleRate,len(aux2)/desiredSampleRate + 1/desiredSampleRate,1/desiredSampleRate)
    # guarda el nombre del canal en channel
    # adquiere el nombre del canal, remueve comillas extras
    channel = data['header']['channel'].strip("'")
    auxaux = codes_df.Area[codes_df.Channel==channel]
    area = auxaux.values[0]
    df2 = pd.DataFrame([[channel,
                         area,
                         orSampleRate,
                         desiredSampleRate,
                         1,
                         data['timestamps'],
                         aux2]],
                       columns=['Channel',
                                'Area',
                                'OriginalSampleRate',
                                'SampleRate',
                                'Use',
                                'Timestamps at original sample rate',
                                'LFP'])
    df = pd.concat([df, df2], axis = 0) 
    #break
df.index = range(0, len(lista))

In [ ]:
df

In [ ]:
# guardar el data frame con los LFP a un archivo pickle
# poner aqui nombre del archivo pickle que se quiere guardar
# nombre sujeto, fecha, condicion, etc.
#file2save = 'RAW_R02_RAT02_SP7_20170111'
#file2save='RAT02_SP_7_2017_01_11_18_46_49'
#file2save = 'EC01_RRR01_SP_2650uA_2017-12-01'
#file2save = 'RC05_SP_10100uA_20171214_170009'
#file2save = 'EC01_RRR03_SP_3500uA_20171215'
#file2save = 'EC01_RRR03_SP_3500uA_L-Dopa_20171215'
#file2save='EC01_RRR03_SP_4800uA_20171221'
#file2save = 'EC01_RRR03_SP_1240uA_20171212'
#file2save = 'E01_RNA03_SP02_500mV_20170923'
#file2save = 'EC01_RR01_SP_6000uA_20171229'

#Cambios Sergio
#file2save = 'EC01_RRR01_SP_12600uA__20180103'
#file2save = 'EC01_RRR03_SP_3500uA_20171215'
#file2save = 'EC01_RC05_SP_10100uA_20171214'
file2save = 'RAT02_SP_7__20170111_184649'
#Para python3, raw_input paso a ser input()
#answer = raw_input('Cambió el nombre del archivo de output? (s/n):')
answer = input('Cambió el nombre del archivo de output? (s/n):')
if answer == 'n':
    print ('Entonces cambielo')
elif answer == 's':
    print ('Guardando...')
    pickle.dump( df, open( rootpath + 
                          'Raw_LFP_' + file2save + '.pickle', "wb" ) )
    # print('Archivo',file2save)
else:
    print ('Entonces cambielo')

In [ ]:
# Guarda archivo con los timestamps del LFP
pickle.dump([orSampleRate,data['timestamps']],
            open( rootpath +
                 'LFP_timestamps_' + file2save + '.pickle', "wb" ) )

In [ ]:
# opcional
# muestra señal filtrada con pasabanda para spikes
N = 2
f1 = 300.0
f2=3000.0
Wn = array([f1/Nyquist, f2/Nyquist])
b,a = scipy.signal.butter(N, Wn, btype='bandpass', analog=False)
for i in range(50,51):
    
    #print i, lista[i]
    # lee el archivo en el i-esimo lugar de la lista y lo coloca en el diccionario data
    data = OE.loadContinuous(lista[i])
    ad = data['data']
    # filter the signal
    adfilt = scipy.signal.filtfilt(b, a, ad, padlen=None)
    fig, ax = subplots()
    auxstr=data['header']['channel'].strip("'")
    #ax.title()
    title(auxstr)
    #ax.plot(ad)
    ax.plot(adfilt,'r')
    ax.set_xlim([0, 600000])
    #break